In [30]:
%pip install langchain langchain_openai langchain_elasticsearch faiss-cpu langchain-community --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 432.1/432.1 kB 1.3 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.9/59.9 kB 1.0 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [1]:
from langchain_community.document_loaders import TextLoader
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.vectorstores.faiss import FAISS

In [32]:
import os
os.environ["OPENAI_API_KEY"] = "API_KEY_HERE"

In [2]:
raw_text = """
Digital marketing encompasses a broad range of marketing activities that utilize digital channels to connect with customers. At its core, digital marketing aims to reach a targeted audience through various online and electronic means, including social media, email, search engines, and websites. Unlike traditional marketing methods, digital marketing offers unparalleled opportunities for businesses to engage with their audience in real-time, enabling personalized communication and immediate feedback. This real-time interaction not only enhances customer experience but also allows businesses to gather valuable data on consumer behaviors, preferences, and trends, facilitating more effective marketing strategies and campaigns.
The rise of digital marketing can be attributed to the increasing reliance on the internet and digital devices by consumers. As more people spend time online, businesses have shifted their marketing efforts to where their audiences are. Digital marketing leverages this online presence, employing strategies such as search engine optimization (SEO), content marketing, pay-per-click (PPC) advertising, and social media marketing to improve visibility and attract potential customers. These strategies are designed to increase traffic to a company's online platforms, build brand awareness, and ultimately drive conversions and sales. The ability to measure the effectiveness of these strategies through analytics and metrics further underscores the advantage of digital marketing, enabling businesses to refine their approach and maximize return on investment (ROI). 
"""

with open("test.txt", "w") as f:
    f.write(raw_text)

In [3]:
# Load the document, split it into chunks, embed each chunk and load it into the vector store.
raw_documents = TextLoader('test.txt').load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
documents = text_splitter.split_documents(raw_documents)
db = FAISS.from_documents(documents, OpenAIEmbeddings())

In [5]:
db.similarity_search_with_relevance_scores("digital marketing", k=1)

[(Document(page_content="Digital marketing encompasses a broad range of marketing activities that utilize digital channels to connect with customers. At its core, digital marketing aims to reach a targeted audience through various online and electronic means, including social media, email, search engines, and websites. Unlike traditional marketing methods, digital marketing offers unparalleled opportunities for businesses to engage with their audience in real-time, enabling personalized communication and immediate feedback. This real-time interaction not only enhances customer experience but also allows businesses to gather valuable data on consumer behaviors, preferences, and trends, facilitating more effective marketing strategies and campaigns.\nThe rise of digital marketing can be attributed to the increasing reliance on the internet and digital devices by consumers. As more people spend time online, businesses have shifted their marketing efforts to where their audiences are. Digi

In [7]:
# Adding on extra documents directly within LangChain:
from langchain_core.documents import Document

docs = [Document(page_content='James phoenix worked in digital marketing for 3 years.', metadata={'source': 'James Phoenix'}),
        Document(page_content='Digital marketing is a growing industry.', metadata={'source': 'Wikipedia'}),
        ]

In [8]:
docs[0].metadata

{'source': 'James Phoenix'}

In [9]:
docs[1].metadata

{'source': 'Wikipedia'}

In [10]:
db.add_documents(docs)

['db785879-a871-4707-9415-bcd650a6b267',
 '6c2a0003-5506-429c-b28b-06928e30880b']

In [11]:
db.similarity_search("James", k=1)

[Document(page_content='James phoenix worked in digital marketing for 3 years.', metadata={'source': 'James Phoenix'})]

---

## Indexing Becomes Messy Without A Record Manager

Without knowing which documents have been indexed, it is difficult to keep track of the documents that have been indexed. This is especially true when the documents are large and the indexing process is time-consuming.

LangChain solves this with an [Indexing API](https://python.langchain.com/docs/modules/data_connection/indexing) that exposes several different methods to keep track of the documents that have been indexed.

`docker run -p 9200:9200 -e "discovery.type=single-node" -e "xpack.security.enabled=false" -e "xpack.security.http.ssl.enabled=false" docker.elastic.co/elasticsearch/elasticsearch:8.9.0`

In [12]:
from langchain.indexes import SQLRecordManager, index
from langchain_elasticsearch import ElasticsearchStore

collection_name = "test_index"

embedding = OpenAIEmbeddings()

vectorstore = ElasticsearchStore(
    es_url="http://localhost:9200", index_name="test_index", embedding=embedding
)

In [18]:
namespace = f"elasticsearch/{collection_name}"
record_manager = SQLRecordManager(
    namespace, db_url="sqlite:///record_manager_cache.sql"
)
record_manager.create_schema()

In [14]:
updated_docs = [
    Document(
        page_content="James phoenix worked in digital marketing for 7 years.",
        metadata={"source": "James Phoenix"},
    ),
   
]

In [19]:
def _clear():
    """Hacky helper method to clear content. See the `full` mode section to to understand why it works."""
    index([], record_manager, vectorstore, cleanup="full", source_id_key="source")

In [21]:
_clear()

In [23]:
docs

[Document(page_content='James phoenix worked in digital marketing for 3 years.', metadata={'source': 'James Phoenix'}),
 Document(page_content='Digital marketing is a growing industry.', metadata={'source': 'Wikipedia'})]

In [25]:
# Indexing all of the documents:
index(
    docs,
    record_manager,
    vectorstore,
    cleanup="incremental",
    source_id_key="source",
)

{'num_added': 0, 'num_updated': 0, 'num_skipped': 2, 'num_deleted': 0}

In [26]:
# Updating a single document:
index(
    updated_docs,
    record_manager,
    vectorstore,
    cleanup="incremental",
    source_id_key="source",
)

{'num_added': 1, 'num_updated': 0, 'num_skipped': 0, 'num_deleted': 1}

In [27]:
# Adding on a new document:
index(
    [Document(page_content="Digital marketing is a growing industry.", metadata={"source": "Wikipedia - Digital Marketing"})],
    record_manager,
    vectorstore,
    cleanup="incremental",
    source_id_key="source",
)

{'num_added': 1, 'num_updated': 0, 'num_skipped': 0, 'num_deleted': 0}

In [28]:
# Skipping documents because the document hash is exactly the same:
index(
    [Document(page_content="Digital marketing is a growing industry.", metadata={"source": "Wikipedia - Digital Marketing"})],
    record_manager,
    vectorstore,
    cleanup="incremental",
    source_id_key="source",
)

{'num_added': 0, 'num_updated': 0, 'num_skipped': 1, 'num_deleted': 0}

## There are 3 modes of using the Index API.


| Cleanup Mode | De-Duplicates Content | Parallelizable | Cleans Up Deleted Source Docs | Cleans Up Mutations of Source Docs and/or Derived Docs | Clean Up Timing      |
|--------------|-----------------------|----------------|-------------------------------|--------------------------------------------------------|----------------------|
| None         | ✅                     | ✅              | ❌                             | ❌                                                      | -                    |
| Incremental  | ✅                     | ✅              | ❌                             | ✅                                                      | Continuously         |
| Full         | ✅                     | ❌              | ✅                             | ✅                                                      | At end of indexing   |


Read more here: https://python.langchain.com/docs/modules/data_connection/indexing